In [2]:
from Bio import SeqIO
from collections import defaultdict, Counter
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# === CAI weights for E. coli K12 (normalized to max=1.0 per amino acid)
CAI_WEIGHTS = {
    'TTT': 0.58, 'TTC': 1.00, 'TTA': 0.02, 'TTG': 0.07,
    'CTT': 0.13, 'CTC': 0.20, 'CTA': 0.02, 'CTG': 1.00,
    'ATT': 0.49, 'ATC': 1.00, 'ATA': 0.03, 'ATG': 1.00,
    'GTT': 0.35, 'GTC': 0.47, 'GTA': 0.07, 'GTG': 1.00,
    'TCT': 0.22, 'TCC': 0.39, 'TCA': 0.15, 'TCG': 0.06,
    'AGT': 0.15, 'AGC': 1.00,
    'CCT': 0.42, 'CCC': 0.29, 'CCA': 0.28, 'CCG': 1.00,
    'ACT': 0.23, 'ACC': 1.00, 'ACA': 0.36, 'ACG': 0.47,
    'GCT': 0.37, 'GCC': 1.00, 'GCA': 0.28, 'GCG': 0.76,
    'TAT': 0.43, 'TAC': 1.00, 'CAT': 0.43, 'CAC': 1.00,
    'CAA': 0.27, 'CAG': 1.00, 'AAT': 0.47, 'AAC': 1.00,
    'AAA': 0.44, 'AAG': 1.00, 'GAT': 0.63, 'GAC': 1.00,
    'GAA': 0.68, 'GAG': 1.00, 'TGT': 0.44, 'TGC': 1.00,
    'TGG': 1.00,
    'CGT': 0.36, 'CGC': 1.00, 'CGA': 0.07, 'CGG': 0.11,
    'AGA': 0.02, 'AGG': 0.02,
    'GGT': 0.41, 'GGC': 1.00, 'GGA': 0.25, 'GGG': 0.50
}

# === tAI weights for E. coli K12 (approximate, dos Reis 2004)
TAI_WEIGHTS = {
    'TTT': 0.43, 'TTC': 1.00, 'TTA': 0.17, 'TTG': 0.32,
    'CTT': 0.22, 'CTC': 0.38, 'CTA': 0.07, 'CTG': 1.00,
    'ATT': 0.38, 'ATC': 0.69, 'ATA': 0.10, 'ATG': 1.00,
    'GTT': 0.31, 'GTC': 0.44, 'GTA': 0.09, 'GTG': 1.00,
    'TCT': 0.32, 'TCC': 0.51, 'TCA': 0.27, 'TCG': 0.23,
    'AGT': 0.25, 'AGC': 0.55,
    'CCT': 0.31, 'CCC': 0.29, 'CCA': 0.25, 'CCG': 1.00,
    'ACT': 0.28, 'ACC': 1.00, 'ACA': 0.38, 'ACG': 0.47,
    'GCT': 0.37, 'GCC': 1.00, 'GCA': 0.29, 'GCG': 0.69,
    'TAT': 0.37, 'TAC': 1.00, 'CAT': 0.41, 'CAC': 1.00,
    'CAA': 0.36, 'CAG': 1.00, 'AAT': 0.48, 'AAC': 1.00,
    'AAA': 0.38, 'AAG': 1.00, 'GAT': 0.54, 'GAC': 1.00,
    'GAA': 0.59, 'GAG': 1.00, 'TGT': 0.45, 'TGC': 1.00,
    'TGG': 1.00,
    'CGT': 0.27, 'CGC': 1.00, 'CGA': 0.09, 'CGG': 0.13,
    'AGA': 0.05, 'AGG': 0.05,
    'GGT': 0.39, 'GGC': 1.00, 'GGA': 0.21, 'GGG': 0.47
}

# === Codon table for ENC
CODON_TABLE = {
    'F': ['TTT', 'TTC'], 'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
    'I': ['ATT', 'ATC', 'ATA'], 'M': ['ATG'], 'V': ['GTT', 'GTC', 'GTA', 'GTG'],
    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], 'P': ['CCT', 'CCC', 'CCA', 'CCG'],
    'T': ['ACT', 'ACC', 'ACA', 'ACG'], 'A': ['GCT', 'GCC', 'GCA', 'GCG'],
    'Y': ['TAT', 'TAC'], 'H': ['CAT', 'CAC'], 'Q': ['CAA', 'CAG'],
    'N': ['AAT', 'AAC'], 'K': ['AAA', 'AAG'], 'D': ['GAT', 'GAC'],
    'E': ['GAA', 'GAG'], 'C': ['TGT', 'TGC'], 'W': ['TGG'],
    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'G': ['GGT', 'GGC', 'GGA', 'GGG']
}

# === Index calculators
def calculate_cai(seq): return calculate_index(seq, CAI_WEIGHTS)
def calculate_tai(seq): return calculate_index(seq, TAI_WEIGHTS)

def calculate_index(seq, weights):
    seq = seq.upper().replace(" ", "").replace("\n", "")
    log_sum, count = 0, 0
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        if codon in weights and weights[codon] > 0:
            log_sum += math.log(weights[codon])
            count += 1
    return math.exp(log_sum / count) if count else 0

def calculate_enc(seq):
    seq = seq.upper().replace(" ", "").replace("\n", "")
    aa_codon_counts = defaultdict(Counter)

    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        for aa, codons in CODON_TABLE.items():
            if codon in codons:
                aa_codon_counts[aa][codon] += 1

    Fk_list = []
    for codons in aa_codon_counts.values():
        k = len(codons)
        if k <= 1:
            continue
        n = sum(codons.values())
        fk = sum((count / n) ** 2 for count in codons.values())
        if n > 1:
            Fk = (n * fk - 1) / (n - 1)
            if Fk > 0:
                Fk_list.append((k, Fk))

    if not Fk_list:
        return 61.0  # No informative codons

    try:
        enc = 2 + sum(k for k, _ in Fk_list) / sum(1 / Fk for _, Fk in Fk_list)
        return enc
    except ZeroDivisionError:
        return 61.0

# === Main function
def compute_codon_metrics(fasta_path):
    records = SeqIO.parse(fasta_path, "fasta")
    results = []
    for record in records:
        seq = str(record.seq)
        results.append({
            "ID": record.id,
            "CAI": round(calculate_cai(seq), 4),
            "ENC": round(calculate_enc(seq), 2),
            "tAI": round(calculate_tai(seq), 4)
        })
    return pd.DataFrame(results)

def plot_cai_tai_boxplots(df, output_path="codon_metrics_boxplot.png"):
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    sns.boxplot(y=df['CAI'], ax=axes[0], color="mediumblue", width=0.3)
    axes[0].set_title("CAI")
    axes[0].set_ylabel("CAI")
    axes[0].set_xticks([])

    sns.boxplot(y=df['tAI'], ax=axes[1], color="purple", width=0.3)
    axes[1].set_title("tAI")
    axes[1].set_ylabel("tAI")
    axes[1].set_xticks([])

    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()
    print(f"Boxplots saved to {output_path}")

df_metrics = compute_codon_metrics(r"C:\projects\data\Laura_test.fa")  # still includes CAI and tAI only
print(df_metrics)
plot_cai_tai_boxplots(df_metrics)

            ID     CAI   ENC     tAI
0   Laura_test  0.8818  3.59  0.8553
1  Laura_test2  0.6684  3.31  0.6102
Boxplots saved to codon_metrics_boxplot.png
